In [ ]:
# ────────────  버전 ①  :  바로 실행되는 최소 코드  ────────────
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1) PDF를 불러올 로더와 자를 기준(청크 길이‧겹침)을 준비
splitter = RecursiveCharacterTextSplitter(chunk_size=1_000, chunk_overlap=100)
loader   = UnstructuredFileLoader("./files/일임형ISA실무지침.pdf")

# 2) PDF → langchain Document → 지정한 길이로 잘라서 docs 리스트에 담기
docs = loader.load_and_split(text_splitter=splitter)

print(f"문서 조각 개수: {len(docs)}")      # 예: 123
print(docs[0].page_content[:200])          # 첫 청크 미리보기


In [ ]:
# PDF 문서 600자 단위 청크 만들기  (버전 ① : 실행용 최소 코드)

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter

# 1) 600글자씩, 100글자 겹치기로 자를 가위 준비
splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

# 2) PDF 파일을 읽어올 로더 준비  ← 경로·파일명만 바뀌었어요!
loader = PyPDFLoader("./files/일임형ISA실무지침.pdf")

# 3) 로드 + 분할
docs = loader.load_and_split(text_splitter=splitter)

print(f"청크 개수: {len(docs)}")
print(docs[0].page_content[:200])


In [ ]:
# PDF → Chroma (OpenAI 임베딩 3-small 사용, 캐시 포함)

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters               import CharacterTextSplitter
from langchain_openai                       import OpenAIEmbeddings
from langchain.embeddings                   import CacheBackedEmbeddings
from langchain_community.vectorstores       import Chroma
from langchain.storage                      import LocalFileStore

cache = LocalFileStore("./.cache")

# 1) 문서 로드·분할
docs = PyPDFLoader("./files/일임형ISA실무지침.pdf") \
         .load_and_split(CharacterTextSplitter(
             separator="\n", chunk_size=600, chunk_overlap=100
         ))

# 2) ➡️ 3-small 임베딩으로 교체 (차원 512 권장)
base_emb = OpenAIEmbeddings(
    model="text-embedding-3-small",
    dimensions=512
)
emb = CacheBackedEmbeddings.from_bytes_store(base_emb, cache)

# 3) Chroma 벡터 DB
vectorstore = Chroma.from_documents(docs, emb)

# 4) 테스트 질의
for i, doc in enumerate(vectorstore.similarity_search(
        "일임형 ISA에서 납입 한도는 얼마인가요?", k=3), 1):
    print(f"\n--- 결과 {i} ---\n{doc.page_content[:300]} …")


In [ ]:
# PDF → FAISS 벡터 DB + RetrievalQA  (버전 ① : 실행용 최소 코드)
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters               import CharacterTextSplitter
from langchain_openai                       import ChatOpenAI, OpenAIEmbeddings
from langchain.embeddings                   import CacheBackedEmbeddings
from langchain_community.vectorstores       import FAISS
from langchain.storage                      import LocalFileStore
from langchain.chains                       import RetrievalQA

# ── 0. LLM & 캐시 ───────────────────────────
llm   = ChatOpenAI()                   # GPT-4o 등 OpenAI 모델
cache = LocalFileStore("./.cache")     # 임베딩 캐시 폴더

# ── 1. PDF 로드 + 분할 ──────────────────────
loader   = PyPDFLoader("./files/일임형ISA실무지침.pdf")
splitter = CharacterTextSplitter(separator="\n", chunk_size=600, chunk_overlap=100)
docs     = loader.load_and_split(text_splitter=splitter)

# ── 2. 임베딩 (캐시 래핑) ────────────────────
base_emb   = OpenAIEmbeddings()
cached_emb = CacheBackedEmbeddings.from_bytes_store(base_emb, cache)

# ── 3. FAISS 벡터스토어 구축 ────────────────
vectorstore = FAISS.from_documents(docs, cached_emb)

# ── 4. RetrievalQA 체인 ▶ map_rerank 방식 ──
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_rerank", #여기에 다양한 방식을 넣을수있음 stuff 나 map_reduce나
    retriever=vectorstore.as_retriever(k=4),   # 상위 4개 문단 후보
)

answer = qa_chain.run("일임형 ISA의 연간 납입 한도는 얼마인가요?")
print("\n[답변]\n", answer)


In [ ]:
# PDF → FAISS 벡터 DB + 맞춤 프롬프트 QA            (하나하나씩 다설명)

# (필요한 모듈 불러오기)
from langchain_community.document_loaders import PyPDFLoader          # PDF 읽는 친구
from langchain_text_splitters               import CharacterTextSplitter   # 글 자르는 가위
from langchain_openai                       import ChatOpenAI, OpenAIEmbeddings # GPT 두뇌 & 숫자 변환 두뇌
from langchain.embeddings                   import CacheBackedEmbeddings   # 두뇌 결과를 기억하는 상자
from langchain_community.vectorstores       import FAISS               # 비슷한 글 찾기 도서관
from langchain.storage                      import LocalFileStore       # 캐시 넣을 폴더
from langchain_core.prompts                 import ChatPromptTemplate   # 챗봇 말버릇 만들기
from langchain_core.runnables               import RunnablePassthrough  # 질문 그대로 pass!

# (0) 캐시 폴더와 GPT 준비
cache_folder = LocalFileStore("./.cache")    # “.cache” 폴더에 기억 저장
llm          = ChatOpenAI(temperature=0.1)   # 말 잘하는 GPT, 차분하게

# (1) PDF 읽고 600글자씩(앞뒤 100글자 겹치며) 자르기
loader   = PyPDFLoader("./files/일임형ISA실무지침.pdf")      # PDF 파일 고르기
splitter = CharacterTextSplitter(
    separator="\n",        # 줄바꿈을 기준으로
    chunk_size=600,        # 한 조각 600글자
    chunk_overlap=100      # 앞과 뒤가 100글자씩 겹쳐요
)
docs = loader.load_and_split(text_splitter=splitter)

# (2) 글 조각을 숫자(벡터)로 바꿔 주는 두뇌 + 캐시
base_emb      = OpenAIEmbeddings()
cached_emb    = CacheBackedEmbeddings.from_bytes_store(base_emb, cache_folder)

# (3) 숫자 벡터를 FAISS 도서관에 보관
vector_db = FAISS.from_documents(docs, cached_emb)

# (4) “어울리는 글 4개만 찾아줘!” 하는 검색 도구(retriever) 만들기
retriever = vector_db.as_retriever(k=4)

# (5) 챗봇에게 “이런 말버릇으로 대답해!” 하고 알려주는 프롬프트
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a helpful assistant. Answer questions using only the following "
     "context. If you don't know the answer, just say you don't know:\n\n{context}"),
    ("human", "{question}")
])

# (6) 질문 → 검색 → 프롬프트 → GPT 순으로 이어붙인 체인
qa_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

# (7) 궁금한 것을 물어보기!
question = "일임형 ISA 계좌의 연간 납입 한도는 얼마인가요?"
print("[질문] ", question)
print("[답변] ", qa_chain.invoke(question))


In [ ]:
# PDF → FAISS + map-reduce QA          

from langchain_community.document_loaders import PyPDFLoader            # PDF 읽는 친구
from langchain_text_splitters               import CharacterTextSplitter     # 글 자르는 가위
from langchain_openai                       import ChatOpenAI, OpenAIEmbeddings # GPT 두뇌 & 숫자 변환 두뇌
from langchain.embeddings                   import CacheBackedEmbeddings     # 두뇌 결과 기억 상자
from langchain_community.vectorstores       import FAISS                 # 비슷한 글 찾기 도서관
from langchain.storage                      import LocalFileStore         # 캐시 폴더
from langchain_core.prompts                 import ChatPromptTemplate     # 챗봇 말버릇
from langchain_core.runnables               import RunnablePassthrough, RunnableLambda

# (0) GPT 와 캐시 준비
llm   = ChatOpenAI(temperature=0.1)           # 차분하게 대답하는 GPT
cache = LocalFileStore("./.cache")            # ".cache" 폴더에 기억 저장

# (1) PDF 읽고 600글자씩(100글자 겹치며) 자르기
loader   = PyPDFLoader("./files/일임형ISA실무지침.pdf")
splitter = CharacterTextSplitter(
    separator="\n", chunk_size=600, chunk_overlap=100
)
docs = loader.load_and_split(text_splitter=splitter)

# (2) 글 조각을 숫자(벡터)로 바꿀 두뇌 + 캐시
base_emb   = OpenAIEmbeddings()
cached_emb = CacheBackedEmbeddings.from_bytes_store(base_emb, cache)

# (3) 숫자 벡터를 FAISS 도서관에 저장
vector_db = FAISS.from_documents(docs, cached_emb)

# (4) “비슷한 글 6개 찾아줘!” 검색 도구
retriever = vector_db.as_retriever(k=6)

# (5) 문단 하나하나가 ‘도움 되는지’ GPT로 점검 (map 단계)
map_doc_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "아래 글이 질문에 도움이 되면 그대로 돌려주고, 아니면 ''만 돌려줘.\n------\n{context}"),
    ("human", "{question}")
])
map_doc_chain = map_doc_prompt | llm

def map_docs(inputs):
    """검색 결과 각 문단을 GPT에게 보여 주고, 도움이 되는 부분만 모아 돌려줘요."""
    docs_   = inputs["documents"]
    quest   = inputs["question"]
    pieces  = [
        map_doc_chain.invoke({"context": d.page_content, "question": quest}).content
        for d in docs_
    ]
    joined  = "\n\n".join(p for p in pieces if p.strip())
    return joined or "''"

map_chain = (
    {"documents": retriever, "question": RunnablePassthrough()}
    | RunnableLambda(map_docs)
)

# (6) 모아온 ‘도움 되는 글’만 보고 최종 답변 (reduce 단계)
final_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "문맥에 있는 정보만 사용해 대답해. 모르면 모른다고 해.\n------\n{context}"),
    ("human", "{question}")
])

qa_chain = (
    {"context": map_chain, "question": RunnablePassthrough()}
    | final_prompt
    | llm
)

# (7) 궁금한 걸 물어보기!
question = "일임형 ISA 관련 법령에서 언급된 ‘납입 한도’는 연간 얼마인가요?"
print("[질문]", question)
print("[답변]", qa_chain.invoke(question))


In [ ]:
# PDF → FAISS + map-reduce QA (신형 임베딩)

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters               import CharacterTextSplitter
from langchain_openai                       import ChatOpenAI, OpenAIEmbeddings
from langchain.embeddings                   import CacheBackedEmbeddings
from langchain_community.vectorstores       import FAISS
from langchain.storage                      import LocalFileStore
from langchain_core.prompts                 import ChatPromptTemplate
from langchain_core.runnables               import RunnablePassthrough, RunnableLambda

llm   = ChatOpenAI(temperature=0.1)
cache = LocalFileStore("./.cache")

# 1) 문서 로드 & 분할
docs = (
    PyPDFLoader("./files/일임형ISA실무지침.pdf")
    .load_and_split(
        CharacterTextSplitter(separator="\n", chunk_size=600, chunk_overlap=100)
    )
)

# 2) 새 임베딩 모델 지정
base_emb = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=512)
emb      = CacheBackedEmbeddings.from_bytes_store(base_emb, cache)

# 3) FAISS 벡터 DB
db        = FAISS.from_documents(docs, emb)
retriever = db.as_retriever(k=6)

# 4) map 단계
map_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "아래 글이 질문에 도움이 되면 그대로 돌려주고, 아니면 ''만 돌려줘.\n------\n{context}"),
    ("human", "{question}")
])
map_chain = map_prompt | llm

def filter_docs(inp):
    q = inp["question"]
    return "\n\n".join(
        map_chain.invoke({"context": d.page_content, "question": q}).content
        for d in inp["documents"]
    ) or "''"

map_pipe = (
    {"documents": retriever, "question": RunnablePassthrough()}
    | RunnableLambda(filter_docs)
)

# 5) reduce 단계
final_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "문맥 정보만 사용해서 답해. 모르면 모른다고 해.\n------\n{context}"),
    ("human", "{question}")
])

qa_chain = (
    {"context": map_pipe, "question": RunnablePassthrough()}
    | final_prompt
    | llm
)

print(qa_chain.invoke("일임형 ISA 관련 법령에서 언급된 ‘납입 한도’는 연간 얼마인가요?"))
